# **Managing the Co-Morbidity Data**

The package and usage of it comes from [https://jackwasey.github.io/icd/](https://jackwasey.github.io/icd/).

## Preamble

In [76]:
.libPaths()

[1] "/Users/mathiasschindler/opt/anaconda3/envs/dataviz/lib/R/library"

In [77]:
#library(devtools)
#install_github("jackwasey/icd")

## Load Data

In [78]:
# Load the data
df_comorb <- read.csv(file = '/Users/mathiasschindler/Library/Mobile Documents/com~apple~CloudDocs/BSE/_T2--CML2/Project1/Data/MIMIC_diagnoses.csv')

# Keep only relevant columns
df_comorb <- subset(df_comorb, select = c('SUBJECT_ID', 'ICD9_CODE') )

# Check data type
class(df_comorb)

# Print head
head(df_comorb)

[1] "data.frame"

,SUBJECT_ID,ICD9_CODE
,<int>,<chr>
1,256,53240
2,256,41071
3,256,53560
4,256,40390
5,256,5859
6,256,04186


## **Load `icd`-Library**

In [79]:
library(icd)

## **Transform Comorbidities Information**

### Co-Morbidities One-Hot Encoded

In [80]:
# Transform using `idb` library
comorb_charlson <- comorbid_charlson(df_comorb)
df_comorb_charlson <- as.data.frame(comorb_charlson)
head(df_comorb_charlson)

,MI,CHF,PVD,Stroke,Dementia,Pulmonary,Rheumatic,PUD,LiverMild,DM,DMcx,Paralysis,Renal,Cancer,LiverSevere,Mets,HIV
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
256,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE
512,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
768,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE
1024,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
66560,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
1280,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [81]:
# Convert logicals to numeric
cols <- sapply(df_comorb_charlson, is.logical)
df_comorb_charlson[,cols] <- lapply(df_comorb_charlson[,cols], as.numeric)
head(df_comorb_charlson)

,MI,CHF,PVD,Stroke,Dementia,Pulmonary,Rheumatic,PUD,LiverMild,DM,DMcx,Paralysis,Renal,Cancer,LiverSevere,Mets,HIV
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
256,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0
512,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
768,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1024,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
66560,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1280,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Charlston Score

In [82]:
# Calculcate charlson score
charlson_score <- charlson(df_comorb)

# Save as dataframe
df_charlson_score <- as.data.frame(charlson_score)
head(df_charlson_score, 15)

,charlson_score
,<dbl>
256,6
512,0
768,5
1024,3
66560,2
1280,1
67072,4
1792,2
2048,0


### Joining Both DataFrames

In [83]:
# Investigate unique SUBJECT_IDs in both dataframes
head(unique(rownames(df_comorb_charlson)), 15)
head(unique(rownames(df_charlson_score)), 15)

[1] "256"   "512"   "768"   "1024"  "66560" "1280"  "67072" "1792"  "2048" 
[10] "2304"  "2560"  "2816"  "68352" "3328"  "3840"

[1] "256"   "512"   "768"   "1024"  "66560" "1280"  "67072" "1792"  "2048" 
[10] "2304"  "2560"  "2816"  "68352" "3328"  "3840"

In [84]:
# Add rownames (pythonic: index) as column
library(tibble)
df_comorb_charlson_idx <- tibble::rownames_to_column(df_comorb_charlson, "subject_id")
df_charlson_score_idx <- tibble::rownames_to_column(df_charlson_score, "subject_id")

head(df_comorb_charlson_idx) 
head(df_charlson_score_idx) 

,subject_id,MI,CHF,PVD,Stroke,Dementia,Pulmonary,Rheumatic,PUD,LiverMild,DM,DMcx,Paralysis,Renal,Cancer,LiverSevere,Mets,HIV
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,256,1,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0
2,512,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,768,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,1024,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
5,66560,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,1280,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,subject_id,charlson_score
,<chr>,<dbl>
1,256,6
2,512,0
3,768,5
4,1024,3
5,66560,2
6,1280,1


In [85]:
comorb_final = merge(df_comorb_charlson_idx, df_charlson_score_idx, by = "subject_id")
head(comorb_final, 15)

,subject_id,MI,CHF,PVD,Stroke,Dementia,Pulmonary,Rheumatic,PUD,LiverMild,DM,DMcx,Paralysis,Renal,Cancer,LiverSevere,Mets,HIV,charlson_score
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,100,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1000,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,5
4,10000,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,6
5,10001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,10002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,10003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,10004,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
9,10005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Export `.csv`

In [86]:
path = "/Users/mathiasschindler/Library/Mobile Documents/com~apple~CloudDocs/BSE/_T2--CML2/Project1/Data/out/"
filename = "comorb_final"
date = format(Sys.time(), "%Y-%m-%d-%H-%M", sep = "")
sep = "-"
filetype = ".csv"

In [87]:
finalfilename = paste(path, filename, sep, date, filetype, sep = "")
finalfilename

[1] "/Users/mathiasschindler/Library/Mobile Documents/com~apple~CloudDocs/BSE/_T2--CML2/Project1/Data/out/comorb_final-2022-02-08-10-52.csv"

In [88]:
write.csv(comorb_final, finalfilename, row.names = TRUE)